## DataLoader

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


class DiabetesDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self, path):
        xy = np.loadtxt(path,
                        delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


dataset = DiabetesDataset(path='./data/diabetes.csv.gz')
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [ ]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

# our model
model = Model()


# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training loop
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.data[0])

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

## GAN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.autograd import Variable

from torchvision import transforms
from torchvision.utils import save_image
from torchvision import datasets

import argparse
import os
import numpy as np
import math

# Make Directory
os.makedirs('result', exist_ok=True)
os.makedirs('mnist', exist_ok=True)

# hyperparameters
parser = argparse.ArgumentParser()
parser.add_argument('--train_iters', type=int, default=200)
parser.add_argument('--batch_size', type=int, default=64)
parser.add_argument('--lr', type=float, default=0.0002)
parser.add_argument('--beta1', type=float, default=0.5)
parser.add_argument('--beta2', type=float, default=0.999)
parser.add_argument('--latent_dim', type=int, default=100)
parser.add_argument('--channels', type=int, default=1)
parser.add_argument('--img_size', type=int, default=28)
parser.add_argument('--sample_step', type=int, default=400)
config = parser.parse_args()
print(config)


img_shape = (config.channels, config.img_size, config.img_size)
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# Data Loader
dataloader = DataLoader(datasets.MNIST('mnist', train=True, download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])),
                        batch_size=config.batch_size, shuffle=True)

In [ ]:
def layer(feat_in, feat_out, bn=True):
    """Custom Linear layer for simplicity."""
    layers = []
    layers.append(nn.Linear(feat_in, feat_out))
    if bn:
        layers.append(nn.BatchNorm1d(feat_out, 0.8))
    return nn.Sequential(*layers)


class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()

        self.layer1 = layer(config.latent_dim, 128, bn=False)
        self.layer2 = layer(128, 256)
        self.layer3 = layer(256, 512)
        self.layer4 = layer(512, 1024)
        self.FC = nn.Linear(1024, int(np.prod(img_shape)))

    def forward(self, x):
        out = F.leaky_relu(self.layer1(x), 0.2)
        out = F.leaky_relu(self.layer2(out), 0.2)
        out = F.leaky_relu(self.layer3(out), 0.2)
        out = F.leaky_relu(self.layer4(out), 0.2)
        out = F.tanh(self.FC(out))

        out = out.view(out.size(0), *img_shape)

        return out

class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()

        self.conv1 = layer(int(np.prod(img_shape)), 512)
        self.conv2 = layer(512, 256)
        self.FC = nn.Linear(256, 1)

    def forward(self, x):
        out = x.view(x.size(0), -1)

        out = F.leaky_relu(self.conv1(out), 0.2)
        out = F.leaky_relu(self.conv2(out), 0.2)
        out = F.sigmoid(self.FC(out))

        return out

In [ ]:
# Loss Function
# Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
Loss_function = nn.BCELoss()

# Initailize Generator and Discriminator
G = G()
D = D()

if cuda:
    G.cuda()
    D.cuda()
    Loss_function.cuda()

# Optimizers
optimizer_G = torch.optim.Adam(G.parameters(), lr=config.lr, betas=(config.beta1, config.beta2))
optimizer_D = torch.optim.Adam(D.parameters(), lr=config.lr, betas=(config.beta1, config.beta2))

#============ Train ============#
for epoch in range(config.train_iters):
    for i, (imgs, _) in enumerate(dataloader):
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)
        real_imgs = Variable(imgs.type(Tensor))

        #============ Train Generator ============#
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], config.latent_dim))))

        # Generate a batch of images
        gen_imgs = G(z)
        # Generator Loss
        loss_G = Loss_function(D(gen_imgs), valid)

        loss_G.backward()
        optimizer_G.step()

        #============ Train Discriminator ============#
        optimizer_D.zero_grad()

        # Discrimnator Loss
        real_loss = Loss_function(D(real_imgs), valid)
        fake_loss = Loss_function(D(gen_imgs.detach()), fake)
        loss_D = (real_loss + fake_loss) / 2

        loss_D.backward()
        optimizer_D.step()

        if i % 100 == 0:
            print("[Epoch %d/%d] [Batch %d/%d] [D loss : %f] [G loss : %f]" % (epoch, config.train_iters, i, len(dataloader), loss_D.item(), loss_G.item()))

        batches_done = epoch * len(dataloader) + i
        if batches_done % config.sample_step == 0:
            save_image(gen_imgs.data[:25], 'result/%d.png' % batches_done, nrow=5, normalize=True)